In [1]:
import numpy as np
import pandas as pd
import os

import csv
import yaml

from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import Namespace

from tqdm import tqdm
from collections import defaultdict

### DRKG Import

In [2]:
drkg_file = 'data/drkg/drkg.tsv'
drkg_df = pd.read_csv(drkg_file, sep="\t", header=None)     # header fix to original code
drkg_triplets_list = drkg_df.values.tolist()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001685D759940>>
Traceback (most recent call last):
  File "c:\Users\cefin\anaconda3\envs\analysis-env\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
  File "c:\Users\cefin\anaconda3\envs\analysis-env\Lib\threading.py", line 1479, in enumerate
    def enumerate():
KeyboardInterrupt: 


### DrugMechDB Import

In [7]:
drugmech_path = "data/drugmech/drugmechdb.yaml"

with open(drugmech_path, 'r') as fh:
        dmdb_yaml = yaml.safe_load(fh)

KeyboardInterrupt: 

In [ ]:
# from dmdb_data_tools_analysis import *

# all_metapath_nodes = get_metapath_node(dmdb_yaml)
# all_metapath_edges = get_metapath_edges(dmdb_yaml)
# basic_stats = defaultdict(list)

# all_metaedges = []
# all_parings = []
# all_targets = []
# unique_metaedges = []
# first_edge_type = []
# all_nodes = []

# id_to_name = {}
# id_to_label = {}

# for i, p in enumerate(dmdb_yaml):
#     _id = (p["graph"]["_id"])
#     drug_id, dis_id = path_to_tup(p)
#     paths = get_all_paths(p)
#     G = path_to_G(p)
    
#     G = add_metaedges(G)
#     G = add_meanode_pairs(G)
    
#     basic_stats['idx'].append(i) #index
#     basic_stats['id'].append(p['graph']['_id']) #DrugMechDB id
#     basic_stats['drug'].append(drug_id) #Drug id
#     basic_stats['disease'].append(dis_id)#Disease id
#     basic_stats['nodes'].append((G.nodes)) #nodes in metapath
#     basic_stats['n_nodes'].append(len(G.nodes)) # number of nodes in metapath
#     basic_stats['n_edges'].append(len(G.edges)) #number of edges in metapath
#     basic_stats['n_paths'].append(len(all_metapath_nodes[_id])) #number of paths
#     basic_stats['metapath'].append(all_metapath_nodes[_id])
#     basic_stats['metapath_with_edges'].append(all_metapath_edges[_id])

    
#     this_metaedges = [G.edges[e]['metaedge'] for e in G.edges]
    
#     all_metaedges += this_metaedges
#     unique_metaedges += list(set(this_metaedges))
    
#     all_parings += [G.edges[e]['mn_pair'] for e in G.edges]
#     all_targets += get_targets(G)
#     first_edge_type += get_target_metaedges(G)
#     all_nodes += list(G.nodes)
    
#     id_to_label = {**id_to_label, **get_id_to_type(G)}
#     id_to_name = {**id_to_name, **get_id_to_name(G)}
    
# dmdb_df = pd.DataFrame(basic_stats)

<!-- ### DruugMechDB Import -->

## Entity Dictionaries

In [ ]:
def insert_entry(entity,ent_type,dic, total_count):
    if ent_type not in dic:
        dic[ent_type]={}
    ent_n_id=len(dic[ent_type])
    if entity not in dic[ent_type]:
        dic[ent_type][entity]=ent_n_id + total_count
    return dic

In [ ]:
drkg_entity_dictionary={}
total_count = 0

for triple in drkg_triplets_list:
    head,pred,tail = triple

    head_type, head = head.split('::')
    tail_type, tail = tail.split('::')
    insert_entry(head,head_type,drkg_entity_dictionary,total_count)
    total_count+=1
    insert_entry(tail,tail_type,drkg_entity_dictionary,total_count)
    total_count+=1

In [ ]:
# Uses MESH and disregards DB for 'Drug' type
dmdb_entity_dictionary = {}
total_count = 0

# DEBUG
counter = 0
debug_set = set()

for entry in dmdb_yaml:
    dbg_flag = 0
    drug_db_id = entry['graph']['drugbank']
    drug_mesh_id = entry['graph']['drug_mesh']       ## Can't trust the database, checking entity class

    for node in entry['nodes']:
        ent_id = node['id']
        ent_type = node['label']
        insert_entry(ent_id,ent_type,dmdb_entity_dictionary,total_count)
        total_count+=1

        if drug_mesh_id is not None and ent_id == drug_mesh_id:      # Extra (duplicate) log with drugbank id (only if drugmesh is not null)
            insert_entry(drug_db_id,ent_type,dmdb_entity_dictionary,total_count)
            total_count+=1
            dbg_flag = 1

        if drug_mesh_id is None:    ## for DEBUG - if null, DB id is used as default
            dbg_flag = 1

    if dbg_flag == 0:   ## for DEBUG
        #print(drug_db_id)
        debug_set.add(drug_db_id)
        counter += 1

print(counter)
print(len(debug_set))   # TODO

116
55


### Inverse Dicts for Entity Search 

In [ ]:
#dmdb_inv_ent_dict = {v: k for k, v in dmdb_entity_dictionary.items()}
dmdb_inverse_ent_dict = {}
for subdict in dmdb_entity_dictionary.values():
    for k, v in subdict.items():
        dmdb_inverse_ent_dict[v]=k


#drkg_inv_ent_dict = {v: k for k, v in drkg_entity_dictionary.items()}
drkg_inverse_ent_dict = {}
for subdict in drkg_entity_dictionary.values():
    for k, v in subdict.items():
        drkg_inverse_ent_dict[v]=k

## Discovering format-defying exceptions

In [ ]:
# TODO: How to handle exceptions

def print_all_type(list,feat):
    for l in list['nodes']:
        print(l[feat])

for entry in dmdb_yaml:
    if entry['nodes'][0]['label'] == 'Drug' and entry['nodes'][-1]['label'] == 'Disease':
        for idx, node in enumerate(entry['nodes']):
            if idx == 0 or idx == (len(entry['nodes'])-1): continue
            elif node['label'] == 'Drug' or node['label'] == 'Disease':
                print_all_type(entry,'label')
                print()
    else:
        print("\t", entry['nodes'][0]['label'], entry['nodes'][-1]['label'])
        print()


### Notes

        drug: Ammonium lactate
        drug_mesh: MESH:null

DrugMechDB contains both  MESH:null and null for drug_mesh, and only one null for drugbank section.

        drug: tazanolast
        drug_mesh: MESH:C106301
        drugbank: null

And only two instances of double entries in drug_mesh:

        drug: acrisorcin
        drug_mesh: MESH:D000585, MESH:D006604
        drugbank: DB:DB11254

        drug: carfilzomib
        drug_mesh: MESH:C524865, MESH:C519125
        drugbank: DB:DB08889

In [22]:
print(len(dmdb_entity_dictionary['Drug']))
print(len(drkg_entity_dictionary['Compound']))
print()
print(len(dmdb_entity_dictionary['Disease']))
print(len(drkg_entity_dictionary['Disease']))

3135
24313

764
5103


In [23]:
def print_sample(dic, key):
    for k,v in dic[key].items():
        print(k)
        break

print_sample(dmdb_entity_dictionary, 'Drug')
print_sample(drkg_entity_dictionary, 'Compound')
print()
print_sample(dmdb_entity_dictionary, 'Disease')
print_sample(drkg_entity_dictionary, 'Disease')

MESH:D000068877
DB02573

MESH:D015464
SARS-CoV2 E


In [25]:
# 1 DrugBank ID is null

cnt = 0
for entry in dmdb_yaml:
    elem = entry['graph']['drugbank']
    if elem is None: 
        print("null")
        continue
    if "DB" in elem:
        cnt += 1
cnt

null


4845

In DRKG;

Compound::DB00898
Compound::MESH:C106876      Disease::MESH:C537014
Compound::MESH:D000588      Disease::MESH:D013274

Compound::MESH:C554292|MESH:D013755


In DMDB;

drugbank: DB:DB09310
drug_mesh: MESH:D000068877  disease_mesh: MESH:C567691

drug_mesh: MESH:C524865, MESH:C519125

# Drugs, Diseases, & Commons

## Drugs

In [ ]:
def insert_entry_eid(entity,ent_type,eid,dic):
    if ent_type not in dic:
        dic[ent_type]={}
    ent_n_id=eid
    if entity not in dic[ent_type]:
        dic[ent_type][entity]=ent_n_id
    return dic

In [ ]:
dmdb_drug_dict = {}

for drug, eid in dmdb_entity_dictionary['Drug'].items():
    if drug != None:
        identifier, idx = drug.split(":")
    insert_entry_eid(idx, identifier, eid, dmdb_drug_dict)

NameError: name 'dmdb_entity_dictionary' is not defined

In [ ]:
dmdb_entity_dictionary['Drug'][None]    # TODO: Not okay?

405

In [ ]:
for k,v in dmdb_drug_dict.items():
    print(len(v),k)#,v)

1556 MESH
1577 DB
1 CHEBI


In [ ]:
drkg_drug_dict = {}

for drug,eid in drkg_entity_dictionary['Compound'].items():
    if len(drug.split(":")) == 1:    # DB and CHEMBL
        if 'DB' in drug:
            insert_entry_eid(drug, 'DB', eid, drkg_drug_dict)
        elif 'CHEMBL' in drug: 
            insert_entry_eid(drug, 'CHEMBL', eid, drkg_drug_dict)
        else:
            print("DEBUG Outlier with no split:", drug)  
    else:
        if "|" in drug:       # Exceptions  --  MESH:C002480|MESH:, MESH:C011440|MESH:C046229,...
            continue
            # TODO: figure how to handle them appropriately
        else:
            identifier, idx = drug.split(":")
            insert_entry_eid(idx, identifier, eid, drkg_drug_dict)

In [ ]:
for k,v in drkg_drug_dict.items():
    print(len(v),k)#,v)

NameError: name 'drkg_drug_dict' is not defined

In [ ]:
# common_drug_set = set()
common_drug_dict = defaultdict(tuple)

counter = [0,0,0]
for mesh_drug,dmdb_id in dmdb_drug_dict['MESH'].items():
    if mesh_drug in drkg_drug_dict['MESH'].keys():
        # common_drug_set.add('MESH:'+ mesh_dis)       # Re-adding identifier 'MESH:'
        common_drug_dict['MESH:'+ mesh_drug] = (dmdb_id,drkg_drug_dict['MESH'][mesh_drug])
        counter[0]+=1
for db_drug,dmdb_id in dmdb_drug_dict['DB'].items():
    if db_drug in drkg_drug_dict['DB'].keys():
        # common_drug_set.add('DB:'+ mesh_dis)       # Re-adding identifier 'DB:'
        common_drug_dict['DB:'+ db_drug] = (dmdb_id,drkg_drug_dict['DB'][db_drug])
        counter[1]+=1
for chebi_drug,dmdb_id in dmdb_drug_dict['CHEBI'].items():
    if chebi_drug in drkg_drug_dict['CHEBI'].keys():
        # common_drug_set.add('CHEBI:'+ mesh_dis)       # Re-adding identifier 'CHEBI:'
        common_drug_dict['CHEBI:'+ chebi_drug] = (dmdb_id,drkg_drug_dict['CHEBI'][chebi_drug])
        counter[2]+=1

print("Number of common drugs [MESH/DB/CHEBI]:", len(common_drug_dict), counter)

KeyError: 'MESH'

## Disease

In [ ]:
dmdb_disease_dict = {}

for disease, eid in dmdb_entity_dictionary['Disease'].items():
    identifier, idx = disease.split(":")
    insert_entry_eid(idx, identifier, eid, dmdb_disease_dict)

In [155]:
for k,v in dmdb_disease_dict.items():
    print(len(v),k)#,v)

758 MESH
5 DB
1 HP


In [156]:
drkg_disease_dict = {}

for disease, eid in drkg_entity_dictionary['Disease'].items():
    if len(disease.split(":")) != 2:
        insert_entry_eid(disease, 'other', eid, drkg_disease_dict)
    else:
        identifier, idx = disease.split(":")
        insert_entry_eid(idx, identifier, eid, drkg_disease_dict)

In [157]:
for k,v in drkg_disease_dict.items():
    print(len(v),k)#,v)

27 other
4871 MESH
78 OMIM
127 DOID


In [158]:
# MESH
# common_disease_set = set()
common_disease_dict = defaultdict(tuple)

for mesh_dis,dmdb_id in dmdb_disease_dict['MESH'].items():
    if mesh_dis in drkg_disease_dict['MESH'].keys():
        # common_disease_set.add('MESH:'+ mesh_dis)       # Re-adding identifier 'MESH:'
        common_disease_dict['MESH:'+ mesh_dis] = (dmdb_id,drkg_disease_dict['MESH'][mesh_dis])

print("Number of common MESH diseases:", len(common_disease_dict))

Number of common MESH diseases: 714


# Common Pairs

TODO : Include entity dictionary IDs for keeping track, use it in paths too. 

In [66]:
# for entry in dmdb_yaml:
#     dis_mesh = entry['graph']['disease_mesh']
#     if len(dis_mesh.split(':')) != 2:
#         print(dis_mesh)

In [ ]:
# Check if both drug and disease elements exist in common entities
# TODO: How is it possible that there are duplicates? (184 duplicates)

# count = [0,0,0]
# common_pairs = set()

# for entry in dmdb_yaml:
#     graph_dict = entry['graph']
#     count[2] += 1
#     db_id = graph_dict['drugbank']
#     mesh_id = graph_dict['drug_mesh']
#     dis_id = graph_dict['disease_mesh']

#     if db_id in common_drug_dict and db_id != None:
#         if dis_id in common_disease_dict:
#             if (db_id, dis_id) in common_pairs:
#                 print("Duplicate", (db_id, dis_id))
#             common_pairs.add((db_id, dis_id))
#             count[0] += 1
#         else: count[1] += 1
#     elif mesh_id in common_drug_dict and mesh_id != None and mesh_id != 'MESH:null':
#         if dis_id in common_disease_dict:
#             if (mesh_id, dis_id) in common_pairs:
#                 print("Duplicate", (mesh_id, dis_id))
#             common_pairs.add((mesh_id, dis_id))
#             count[0] += 1
#         else: count[1] += 1
#     else:
#         count[1] += 1


# print("Number of common drug-disease pairs:", len(common_pairs))
# print("[DB, MESH, Total]", count)

# Number of common drug-disease pairs: 4484
# [DB, MESH, Total] [4668, 178, 4846]

In [160]:
common_pairs_dict = defaultdict()

for entry in dmdb_yaml:
    graph_dict = entry['graph']
    db_id = graph_dict['drugbank']
    mesh_id = graph_dict['drug_mesh']
    dis_id = graph_dict['disease_mesh']

    if db_id in common_drug_dict and db_id != None:     # Drugbank ID
        if dis_id in common_disease_dict:
            common_pairs_dict[(db_id,dis_id)] = 'DB' # = ((dmdb_drug_dict['DB'][db_id],drkg_drug_dict['DB'][db_id]),(dmdb_disease_dict['MESH'][dis_id],drkg_disease_dict['MESH'][dis_id]))
    elif mesh_id in common_drug_dict and mesh_id != None and mesh_id != 'MESH:null':    # Drug Mesh ID  - no duplicates (Drugbank ID has priority)
        if dis_id in common_disease_dict:
            common_pairs_dict[(mesh_id,dis_id)] = 'MESH'

print("Number of common drug-disease pairs:", len(common_pairs_dict),'/',len(dmdb_yaml))

Number of common drug-disease pairs: 4484 / 4846


In [161]:
# Testing reverse pair search
for common_pair in common_pairs_dict.keys():
    common_drug, common_disease = common_pair
    d_type, drug = common_drug.split(':')
    dis_type, disease = common_disease.split(':')
    print("(DRUG) DMDB Entity index, DRKG Entity index:", dmdb_drug_dict[d_type][drug], drkg_drug_dict[d_type][drug])
    print("(DISEASE) DMDB Entity index, DRKG Entity index:", dmdb_disease_dict[dis_type][disease], drkg_disease_dict[dis_type][disease])
    print()
    break

(DRUG) DMDB Entity index, DRKG Entity index: 2 169474
(DISEASE) DMDB Entity index, DRKG Entity index: 3 3039657



# Pathing strategy for DrugMechDB

Determine the data structure formatting to store the subgraph structure. Use integer IDs in entity dictionaries and focus on connections.

# GNN Predictions

In [ ]:
# Import trained GNNs


In [ ]:
# Format inputs appropriately (care splitting training/eval/test datasets)


## Saliency 

In [ ]:
# Get mask nodes from SM

## Subgraph Comparison

In [ ]:
# Compare prediction paths of Saliency Maps and ground-truth DrugMechDB subgraph samples